In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Dense 
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder 
from sklearn.model_selection import train_test_split 
df = pd.read_csv("./dataset/pima-indians-diabetes.csv", header=None) 
x = df.values[:,0:8] 
y = df.values[:,8]
np.unique(y, return_counts=True)

(array([0., 1.]), array([500, 268], dtype=int64))

In [2]:
x_train, x_test, y_train, y_test = train_test_split(x, y, stratify=y,
                                                    test_size=0.3, 
                                                    random_state=789)
print(x_train.shape, y_train.shape)
print(np.unique(y_train, return_counts=True))
print(x_train[:2,:])

(537, 8) (537,)
(array([0., 1.]), array([350, 187], dtype=int64))
[[  2.     71.     70.     27.      0.     28.      0.586  22.   ]
 [  1.    199.     76.     43.      0.     42.9     1.394  22.   ]]


In [3]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x_train)
x_tr_std = scaler.transform(x_train)
x_te_std = scaler.transform(x_test)

In [4]:
print(x_train[:2,:], x_tr_std[:2,:])

[[  2.     71.     70.     27.      0.     28.      0.586  22.   ]
 [  1.    199.     76.     43.      0.     42.9     1.394  22.   ]] [[-0.56142403 -1.59847063  0.07592301  0.40609133 -0.69103384 -0.47265572
   0.295433   -0.94333269]
 [-0.85187172  2.45870884  0.38440211  1.4260085  -0.69103384  1.30612243
   2.63222749 -0.94333269]]


In [5]:
model = Sequential()
model.add(Dense(20, input_dim=8, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(20, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                180       
_________________________________________________________________
dense_1 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 1,041
Trainable params: 1,041
Non-trainable params: 0
_________________________________________________________________


In [6]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping_cb = EarlyStopping(monitor='val_loss', patience=10)
check_pt = ModelCheckpoint(filepath="./pima-std-model.hdf5",
                           monitor='val_loss', verbose=1, save_best_only=True)
model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=1000, 
          batch_size=200, callbacks=[early_stopping_cb, check_pt])

Epoch 1/1000
1/3 [=========>....................] - ETA: 0s - loss: 1.4806 - accuracy: 0.5750
Epoch 00001: val_loss improved from inf to 1.11819, saving model to .\pima-std-model.hdf5
3/3 [==============================] - 0s 77ms/step - loss: 1.5229 - accuracy: 0.5717 - val_loss: 1.1182 - val_accuracy: 0.6494
Epoch 2/1000
1/3 [=========>....................] - ETA: 0s - loss: 1.2592 - accuracy: 0.5900
Epoch 00002: val_loss improved from 1.11819 to 0.96589, saving model to .\pima-std-model.hdf5
3/3 [==============================] - 0s 12ms/step - loss: 1.2478 - accuracy: 0.5791 - val_loss: 0.9659 - val_accuracy: 0.6407
Epoch 3/1000
1/3 [=========>....................] - ETA: 0s - loss: 1.2462 - accuracy: 0.5850
Epoch 00003: val_loss improved from 0.96589 to 0.93131, saving model to .\pima-std-model.hdf5
3/3 [==============================] - 0s 11ms/step - loss: 1.1129 - accuracy: 0.5903 - val_loss: 0.9313 - val_accuracy: 0.6147
Epoch 4/1000
1/3 [=========>....................] - ETA:

Epoch 28/1000
1/3 [=========>....................] - ETA: 0s - loss: 0.5253 - accuracy: 0.7600
Epoch 00028: val_loss did not improve from 0.61182
3/3 [==============================] - 0s 5ms/step - loss: 0.5568 - accuracy: 0.7281 - val_loss: 0.6230 - val_accuracy: 0.6580
Epoch 29/1000
1/3 [=========>....................] - ETA: 0s - loss: 0.5863 - accuracy: 0.7250
Epoch 00029: val_loss did not improve from 0.61182
3/3 [==============================] - 0s 5ms/step - loss: 0.5641 - accuracy: 0.7188 - val_loss: 0.6233 - val_accuracy: 0.6710
Epoch 30/1000
1/3 [=========>....................] - ETA: 0s - loss: 0.5846 - accuracy: 0.6950
Epoch 00030: val_loss improved from 0.61182 to 0.60961, saving model to .\pima-std-model.hdf5
3/3 [==============================] - 0s 10ms/step - loss: 0.5599 - accuracy: 0.7244 - val_loss: 0.6096 - val_accuracy: 0.6710
Epoch 31/1000
1/3 [=========>....................] - ETA: 0s - loss: 0.5420 - accuracy: 0.7350
Epoch 00031: val_loss did not improve from

In [7]:
from tensorflow.keras.models import load_model

ld_model = load_model("./pima-std-model.hdf5")
ld_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 20)                180       
_________________________________________________________________
dense_1 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_2 (Dense)              (None, 20)                420       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 21        
Total params: 1,041
Trainable params: 1,041
Non-trainable params: 0
_________________________________________________________________


In [8]:
ld_model.evaluate(x_test, y_test)

8/8 [==============================] - 0s 1ms/step - loss: 0.6054 - accuracy: 0.6753


[0.6053667664527893, 0.6753246784210205]

In [9]:
from konlpy.tag import Okt

In [10]:
okt = Okt()
print(okt.morphs("열심히 코딩한 당신. 연휴에는 여행을 가봐요~"))

['열심히', '코딩', '한', '당신', '.', '연휴', '에는', '여행', '을', '가봐요', '~']


In [11]:
text = "You say goodbay, and I say hello." 
text = text.lower() 
text = text.replace(',', '')
text = text.replace('.', '')
text

'you say goodbay and i say hello'

In [12]:
words = text.split(' ')
words

['you', 'say', 'goodbay', 'and', 'i', 'say', 'hello']

In [13]:
word_to_id = {w:i for i, w in enumerate(set(words))}
word_to_id

{'i': 0, 'hello': 1, 'goodbay': 2, 'say': 3, 'you': 4, 'and': 5}

In [14]:
corpus = [word_to_id[w] for w in words]
corpus

[4, 3, 2, 5, 0, 3, 1]

In [15]:
import nltk
nltk.download('all-corpora')
nltk.download('punkt')

[nltk_data] Downloading collection 'all-corpora'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\etriai08\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\etriai08\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     C:\Users\etriai08\AppData\Roaming\nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     C:\Users\etriai08\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     C:\Users\etriai08\AppData\Roaming\nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_da

[nltk_data]    |   Package ptb is already up-to-date!
[nltk_data]    | Downloading package qc to
[nltk_data]    |     C:\Users\etriai08\AppData\Roaming\nltk_data...
[nltk_data]    |   Package qc is already up-to-date!
[nltk_data]    | Downloading package reuters to
[nltk_data]    |     C:\Users\etriai08\AppData\Roaming\nltk_data...
[nltk_data]    |   Package reuters is already up-to-date!
[nltk_data]    | Downloading package rte to
[nltk_data]    |     C:\Users\etriai08\AppData\Roaming\nltk_data...
[nltk_data]    |   Package rte is already up-to-date!
[nltk_data]    | Downloading package semcor to
[nltk_data]    |     C:\Users\etriai08\AppData\Roaming\nltk_data...
[nltk_data]    |   Package semcor is already up-to-date!
[nltk_data]    | Downloading package senseval to
[nltk_data]    |     C:\Users\etriai08\AppData\Roaming\nltk_data...
[nltk_data]    |   Package senseval is already up-to-date!
[nltk_data]    | Downloading package sentence_polarity to
[nltk_data]    |     C:\Users\etriai

True

In [16]:
from nltk.tokenize import word_tokenize
sentence = "Natural language processing (NLP) is a subfield of computer science. So now can I have lbububu"
print(word_tokenize(sentence))

['Natural', 'language', 'processing', '(', 'NLP', ')', 'is', 'a', 'subfield', 'of', 'computer', 'science', '.', 'So', 'now', 'can', 'I', 'have', 'lbububu']


In [17]:
text2 = "Ph.D. Rock n  roll New York AT&T Don't $4.55 04/05/22 I'm Mr. Jone 129.254.23.13 "
print(word_tokenize(text2))

['Ph.D.', 'Rock', 'n', 'roll', 'New', 'York', 'AT', '&', 'T', 'Do', "n't", '$', '4.55', '04/05/22', 'I', "'m", 'Mr.', 'Jone', '129.254.23.13']


In [18]:
from nltk.tokenize import word_tokenize
sentence = "I am a boy! 나는 소년이다."
print(word_tokenize(sentence))


['I', 'am', 'a', 'boy', '!', '나는', '소년이다', '.']


In [19]:
import spacy

In [20]:
!python -m spacy download en_core_web_sm

[+] Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [21]:
spacy_nlp = spacy.load("en_core_web_sm")
sentence = "Natural language processing (NLP) is a subfield of computer science. So now can I have lbububu"
text = "Ph.D. Rock n roll New York AT&T Don't $4.55 04/05/22 I'm Mr. Jone 129.254.23.13 "

In [22]:
doc = spacy_nlp(sentence)
print([token.text for token in doc])

['Natural', 'language', 'processing', '(', 'NLP', ')', 'is', 'a', 'subfield', 'of', 'computer', 'science', '.', 'So', 'now', 'can', 'I', 'have', 'lbububu']


In [23]:
doc = spacy_nlp(text)
print([token.text for token in doc])

['Ph.D.', 'Rock', 'n', 'roll', 'New', 'York', 'AT&T', 'Do', "n't", '$', '4.55', '04/05/22', 'I', "'m", 'Mr.', 'Jone', '129.254.23.13']


In [24]:
print(word_tokenize(sentence))

['Natural', 'language', 'processing', '(', 'NLP', ')', 'is', 'a', 'subfield', 'of', 'computer', 'science', '.', 'So', 'now', 'can', 'I', 'have', 'lbububu']


In [25]:
print(word_tokenize(text))

['Ph.D.', 'Rock', 'n', 'roll', 'New', 'York', 'AT', '&', 'T', 'Do', "n't", '$', '4.55', '04/05/22', 'I', "'m", 'Mr.', 'Jone', '129.254.23.13']


In [26]:
hangul = "제가이렇게스페이스를사용하지않아도이해되죠?"
eng = "tobeornottobethatisaquestion"

In [27]:
text = "한글 자연어 처리는 재밌다. 이제부터 열심히 해봐야지~ ㅎㅎㅎ"
print(okt.morphs(text)) #형태소 분석해서 나누기 (tokenize)
print(okt.morphs(text, stem=True)) #어근 True
print(okt.nouns(text)) #명사
print(okt.phrases(text)) #명사구
print(okt.pos(text)) #tokenize 뿐만 아니라 품사까지 튜플형태로 찍어줌 (particle은 완전한 문장 아닌거. 분자???)

['한글', '자연어', '처리', '는', '재밌다', '.', '이제', '부터', '열심히', '해봐야지', '~', 'ㅎㅎㅎ']
['한글', '자연어', '처리', '는', '재밌다', '.', '이제', '부터', '열심히', '해보다', '~', 'ㅎㅎㅎ']
['한글', '자연어', '처리', '이제']
['한글', '한글 자연어', '한글 자연어 처리', '이제', '자연어', '처리']
[('한글', 'Noun'), ('자연어', 'Noun'), ('처리', 'Noun'), ('는', 'Josa'), ('재밌다', 'Adjective'), ('.', 'Punctuation'), ('이제', 'Noun'), ('부터', 'Josa'), ('열심히', 'Adverb'), ('해봐야지', 'Verb'), ('~', 'Punctuation'), ('ㅎㅎㅎ', 'KoreanParticle')]


In [34]:
import nltk
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\etriai08\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [35]:
from nltk.tag import pos_tag
sentence = "Natural language processing (NLP) is a subfield of computer science. So now can I have lbububu"
words = word_tokenize(sentence)
pos_tag(words)

[('Natural', 'JJ'),
 ('language', 'NN'),
 ('processing', 'NN'),
 ('(', '('),
 ('NLP', 'NNP'),
 (')', ')'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('subfield', 'NN'),
 ('of', 'IN'),
 ('computer', 'NN'),
 ('science', 'NN'),
 ('.', '.'),
 ('So', 'RB'),
 ('now', 'RB'),
 ('can', 'MD'),
 ('I', 'PRP'),
 ('have', 'VBP'),
 ('lbububu', 'VBN')]

In [36]:
import re # regular expression

In [37]:
re.split("(\w+)", "Wow, it is awesome")

['', 'Wow', ', ', 'it', ' ', 'is', ' ', 'awesome', '']

In [41]:
r = re.compile("a.c") # . 은 반드시 단어 하나 있어야 함
r.search("kkk ac")

In [46]:
r = re.compile("a?c") # ? 는 앞에 문자가 하나 있거나 없거나. 즉, a가 0개 OR 1개.
r.search("kkk bc")

<re.Match object; span=(5, 6), match='c'>

In [49]:
r = re.compile("a*c") # *은 a가 0개 이상
r.search("kkk aaaaaaaac")

<re.Match object; span=(4, 13), match='aaaaaaaac'>

In [51]:
text = """이름: 김철수
전화번호: 010 - 1234 - 5678
나이: 30
성별: 남"""

In [55]:
r = re.compile("[0-9]+")
r.findall(text)

['010', '1234', '5678', '30']

In [58]:
print(re.findall("\d+", text)) #숫자
print(re.findall("\D+", text)) #숫자가 아닌거
print(re.findall("\w+", text)) #문자
print(re.findall("\W+", text)) #문자가 아닌거


['010', '1234', '5678', '30']
['이름: 김철수\n전화번호: ', ' - ', ' - ', '\n나이: ', '\n성별: 남']
['이름', '김철수', '전화번호', '010', '1234', '5678', '나이', '30', '성별', '남']
[': ', '\n', ': ', ' - ', ' - ', '\n', ': ', '\n', ': ']


In [62]:
text = "Regular expression : A regular expression, regex or regexp[1] (someties called a rational expression[2][3] is.)"
pred_text = re.sub('[^a-zA-Z]',' ',text)
pred_text

'Regular expression   A regular expression  regex or regexp     someties called a rational expression       is  '

In [63]:
pred_text.split(" ")

['Regular',
 'expression',
 '',
 '',
 'A',
 'regular',
 'expression',
 '',
 'regex',
 'or',
 'regexp',
 '',
 '',
 '',
 '',
 'someties',
 'called',
 'a',
 'rational',
 'expression',
 '',
 '',
 '',
 '',
 '',
 '',
 'is',
 '',
 '']

In [64]:
re.split("\s+", pred_text)

['Regular',
 'expression',
 'A',
 'regular',
 'expression',
 'regex',
 'or',
 'regexp',
 'someties',
 'called',
 'a',
 'rational',
 'expression',
 'is',
 '']

In [94]:
from sklearn.feature_extraction.text import CountVectorizer

text_data = ["나는 배가 고프다",  # 이거 하나가 도큐먼트
             "내일 점심 뭐먹지",
             "내일 공부 해야겠다",
             "점심 먹고 공부 해야지"]
count_vectorizer = CountVectorizer()
count_vectorizer.fit(text_data)
print(count_vectorizer.vocabulary_)

{'나는': 2, '배가': 6, '고프다': 0, '내일': 3, '점심': 7, '뭐먹지': 5, '공부': 1, '해야겠다': 8, '먹고': 4, '해야지': 9}


In [95]:
sentence = [text_data[0]]
print(count_vectorizer.transform(sentence).toarray())

[[1 0 1 0 0 0 1 0 0 0]]


In [96]:
sentence = ["나는 나는 나는 배가 배가 고프다"]
print(count_vectorizer.transform(sentence).toarray())

[[1 0 3 0 0 0 2 0 0 0]]


In [97]:
print(count_vectorizer.transform(text_data).toarray())

[[1 0 1 0 0 0 1 0 0 0]
 [0 0 0 1 0 1 0 1 0 0]
 [0 1 0 1 0 0 0 0 1 0]
 [0 1 0 0 1 0 0 1 0 1]]


In [98]:
x = np.array(count_vectorizer.transform(text_data).toarray())
y = np.array([0,0,1,1]) #먹는거, 먹는거, 공부하는거, 공부하는거 (문장의 어감이)

model = Sequential()
model.add(Dense(10, input_dim=10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_21 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_22 (Dense)             (None, 10)                110       
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 11        
Total params: 341
Trainable params: 341
Non-trainable params: 0
_________________________________________________________________


In [117]:
model.fit(x, y, epochs=1000)

Epoch 1/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 2/1000
1/1 [==============================] - 0s 999us/step - loss: 0.0016 - accuracy: 1.0000
Epoch 3/1000
1/1 [==============================] - 0s 996us/step - loss: 0.0016 - accuracy: 1.0000
Epoch 4/1000
1/1 [==============================] - 0s 0s/step - loss: 0.0016 - accuracy: 1.0000
Epoch 5/1000
1/1 [==============================] - 0s 998us/step - loss: 0.0016 - accuracy: 1.0000
Epoch 6/1000
1/1 [==============================] - 0s 998us/step - loss: 0.0016 - accuracy: 1.0000
Epoch 7/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0016 - accuracy: 1.0000
Epoch 8/1000
1/1 [==============================] - 0s 0s/step - loss: 0.0016 - accuracy: 1.0000
Epoch 9/1000
1/1 [==============================] - 0s 997us/step - loss: 0.0016 - accuracy: 1.0000
Epoch 10/1000
1/1 [==============================] - 0s 999us/step - loss: 0.0016 - accuracy: 1.0000
Epoch 11/

1/1 [==============================] - 0s 0s/step - loss: 0.0012 - accuracy: 1.0000
Epoch 84/1000
1/1 [==============================] - 0s 997us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 85/1000
1/1 [==============================] - 0s 1ms/step - loss: 0.0012 - accuracy: 1.0000
Epoch 86/1000
1/1 [==============================] - 0s 999us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 87/1000
1/1 [==============================] - 0s 1000us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 88/1000
1/1 [==============================] - 0s 0s/step - loss: 0.0012 - accuracy: 1.0000
Epoch 89/1000
1/1 [==============================] - 0s 996us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 90/1000
1/1 [==============================] - 0s 998us/step - loss: 0.0012 - accuracy: 1.0000
Epoch 91/1000
1/1 [==============================] - 0s 0s/step - loss: 0.0012 - accuracy: 1.0000
Epoch 92/1000
1/1 [==============================] - 0s 0s/step - loss: 0.0012 - accuracy: 1.0000
Epoch 93/1000
1/1

1/1 [==============================] - 0s 1ms/step - loss: 9.4114e-04 - accuracy: 1.0000
Epoch 164/1000
1/1 [==============================] - 0s 1ms/step - loss: 9.3835e-04 - accuracy: 1.0000
Epoch 165/1000
1/1 [==============================] - 0s 1ms/step - loss: 9.3561e-04 - accuracy: 1.0000
Epoch 166/1000
1/1 [==============================] - 0s 996us/step - loss: 9.3289e-04 - accuracy: 1.0000
Epoch 167/1000
1/1 [==============================] - 0s 0s/step - loss: 9.3017e-04 - accuracy: 1.0000
Epoch 168/1000
1/1 [==============================] - 0s 999us/step - loss: 9.2746e-04 - accuracy: 1.0000
Epoch 169/1000
1/1 [==============================] - 0s 1ms/step - loss: 9.2477e-04 - accuracy: 1.0000
Epoch 170/1000
1/1 [==============================] - 0s 1ms/step - loss: 9.2206e-04 - accuracy: 1.0000
Epoch 171/1000
1/1 [==============================] - 0s 1ms/step - loss: 9.1939e-04 - accuracy: 1.0000
Epoch 172/1000
1/1 [==============================] - 0s 0s/step - loss: 9.1

1/1 [==============================] - 0s 0s/step - loss: 7.5719e-04 - accuracy: 1.0000
Epoch 242/1000
1/1 [==============================] - 0s 997us/step - loss: 7.5518e-04 - accuracy: 1.0000
Epoch 243/1000
1/1 [==============================] - 0s 1ms/step - loss: 7.5317e-04 - accuracy: 1.0000
Epoch 244/1000
1/1 [==============================] - 0s 0s/step - loss: 7.5120e-04 - accuracy: 1.0000
Epoch 245/1000
1/1 [==============================] - 0s 1ms/step - loss: 7.4921e-04 - accuracy: 1.0000
Epoch 246/1000
1/1 [==============================] - 0s 998us/step - loss: 7.4724e-04 - accuracy: 1.0000
Epoch 247/1000
1/1 [==============================] - 0s 0s/step - loss: 7.4527e-04 - accuracy: 1.0000
Epoch 248/1000
1/1 [==============================] - 0s 1ms/step - loss: 7.4331e-04 - accuracy: 1.0000
Epoch 249/1000
1/1 [==============================] - 0s 997us/step - loss: 7.4136e-04 - accuracy: 1.0000
Epoch 250/1000
1/1 [==============================] - 0s 997us/step - loss: 

1/1 [==============================] - 0s 9ms/step - loss: 6.2128e-04 - accuracy: 1.0000
Epoch 320/1000
1/1 [==============================] - 0s 0s/step - loss: 6.1978e-04 - accuracy: 1.0000
Epoch 321/1000
1/1 [==============================] - 0s 998us/step - loss: 6.1826e-04 - accuracy: 1.0000
Epoch 322/1000
1/1 [==============================] - 0s 0s/step - loss: 6.1678e-04 - accuracy: 1.0000
Epoch 323/1000
1/1 [==============================] - 0s 996us/step - loss: 6.1528e-04 - accuracy: 1.0000
Epoch 324/1000
1/1 [==============================] - 0s 0s/step - loss: 6.1379e-04 - accuracy: 1.0000
Epoch 325/1000
1/1 [==============================] - 0s 1ms/step - loss: 6.1232e-04 - accuracy: 1.0000
Epoch 326/1000
1/1 [==============================] - 0s 0s/step - loss: 6.1085e-04 - accuracy: 1.0000
Epoch 327/1000
1/1 [==============================] - 0s 996us/step - loss: 6.0937e-04 - accuracy: 1.0000
Epoch 328/1000
1/1 [==============================] - 0s 1ms/step - loss: 6.0

...

1/1 [==============================] - 0s 999us/step - loss: 3.2322e-04 - accuracy: 1.0000
Epoch 632/1000
1/1 [==============================] - 0s 1000us/step - loss: 3.2263e-04 - accuracy: 1.0000
Epoch 633/1000
1/1 [==============================] - 0s 0s/step - loss: 3.2204e-04 - accuracy: 1.0000
Epoch 634/1000
1/1 [==============================] - 0s 999us/step - loss: 3.2145e-04 - accuracy: 1.0000
Epoch 635/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.2087e-04 - accuracy: 1.0000
Epoch 636/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.2029e-04 - accuracy: 1.0000
Epoch 637/1000
1/1 [==============================] - 0s 0s/step - loss: 3.1971e-04 - accuracy: 1.0000
Epoch 638/1000
1/1 [==============================] - 0s 0s/step - loss: 3.1913e-04 - accuracy: 1.0000
Epoch 639/1000
1/1 [==============================] - 0s 1ms/step - loss: 3.1855e-04 - accuracy: 1.0000
Epoch 640/1000
1/1 [==============================] - 0s 1ms/step - loss: 3

1/1 [==============================] - 0s 999us/step - loss: 2.8145e-04 - accuracy: 1.0000
Epoch 710/1000
1/1 [==============================] - 0s 999us/step - loss: 2.8097e-04 - accuracy: 1.0000
Epoch 711/1000
1/1 [==============================] - 0s 997us/step - loss: 2.8049e-04 - accuracy: 1.0000
Epoch 712/1000
1/1 [==============================] - 0s 0s/step - loss: 2.8001e-04 - accuracy: 1.0000
Epoch 713/1000
1/1 [==============================] - 0s 0s/step - loss: 2.7953e-04 - accuracy: 1.0000
Epoch 714/1000
1/1 [==============================] - 0s 0s/step - loss: 2.7905e-04 - accuracy: 1.0000
Epoch 715/1000
1/1 [==============================] - 0s 995us/step - loss: 2.7857e-04 - accuracy: 1.0000
Epoch 716/1000
1/1 [==============================] - 0s 0s/step - loss: 2.7809e-04 - accuracy: 1.0000
Epoch 717/1000
1/1 [==============================] - 0s 0s/step - loss: 2.7761e-04 - accuracy: 1.0000
Epoch 718/1000
1/1 [==============================] - 0s 0s/step - loss: 2.7

1/1 [==============================] - 0s 0s/step - loss: 2.4693e-04 - accuracy: 1.0000
Epoch 788/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.4653e-04 - accuracy: 1.0000
Epoch 789/1000
1/1 [==============================] - 0s 997us/step - loss: 2.4612e-04 - accuracy: 1.0000
Epoch 790/1000
1/1 [==============================] - 0s 997us/step - loss: 2.4572e-04 - accuracy: 1.0000
Epoch 791/1000
1/1 [==============================] - 0s 996us/step - loss: 2.4532e-04 - accuracy: 1.0000
Epoch 792/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.4492e-04 - accuracy: 1.0000
Epoch 793/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.4452e-04 - accuracy: 1.0000
Epoch 794/1000
1/1 [==============================] - 0s 0s/step - loss: 2.4413e-04 - accuracy: 1.0000
Epoch 795/1000
1/1 [==============================] - 0s 0s/step - loss: 2.4373e-04 - accuracy: 1.0000
Epoch 796/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.

1/1 [==============================] - 0s 996us/step - loss: 2.1806e-04 - accuracy: 1.0000
Epoch 866/1000
1/1 [==============================] - 0s 0s/step - loss: 2.1772e-04 - accuracy: 1.0000
Epoch 867/1000
1/1 [==============================] - 0s 0s/step - loss: 2.1739e-04 - accuracy: 1.0000
Epoch 868/1000
1/1 [==============================] - 0s 0s/step - loss: 2.1705e-04 - accuracy: 1.0000
Epoch 869/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.1671e-04 - accuracy: 1.0000
Epoch 870/1000
1/1 [==============================] - 0s 999us/step - loss: 2.1638e-04 - accuracy: 1.0000
Epoch 871/1000
1/1 [==============================] - 0s 0s/step - loss: 2.1604e-04 - accuracy: 1.0000
Epoch 872/1000
1/1 [==============================] - 0s 0s/step - loss: 2.1571e-04 - accuracy: 1.0000
Epoch 873/1000
1/1 [==============================] - 0s 1ms/step - loss: 2.1537e-04 - accuracy: 1.0000
Epoch 874/1000
1/1 [==============================] - 0s 998us/step - loss: 2.15

1/1 [==============================] - 0s 0s/step - loss: 1.9370e-04 - accuracy: 1.0000
Epoch 944/1000
1/1 [==============================] - 0s 0s/step - loss: 1.9341e-04 - accuracy: 1.0000
Epoch 945/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.9312e-04 - accuracy: 1.0000
Epoch 946/1000
1/1 [==============================] - 0s 999us/step - loss: 1.9284e-04 - accuracy: 1.0000
Epoch 947/1000
1/1 [==============================] - 0s 1000us/step - loss: 1.9255e-04 - accuracy: 1.0000
Epoch 948/1000
1/1 [==============================] - 0s 0s/step - loss: 1.9227e-04 - accuracy: 1.0000
Epoch 949/1000
1/1 [==============================] - 0s 0s/step - loss: 1.9198e-04 - accuracy: 1.0000
Epoch 950/1000
1/1 [==============================] - 0s 0s/step - loss: 1.9170e-04 - accuracy: 1.0000
Epoch 951/1000
1/1 [==============================] - 0s 0s/step - loss: 1.9142e-04 - accuracy: 1.0000
Epoch 952/1000
1/1 [==============================] - 0s 1ms/step - loss: 1.9113

In [118]:
test = np.array(count_vectorizer.transform(["나는 뭐먹지"]).toarray())
test

array([[0, 0, 1, 0, 0, 1, 0, 0, 0, 0]], dtype=int64)

In [119]:
model.predict(test)

array([[0.00570694]], dtype=float32)

In [130]:
from sklearn.feature_extraction.text import CountVectorizer

text_data = ["나는 배가 고프다 너는 배가 고프냐",  # 이거 하나가 도큐먼트
             "내일 점심 뭐먹지 너는 뭐먹지",
             "내일 공부 해야겠다 내일 공부 해야지",
             "점심 먹고 공부 해야지 너는 공부 해야지"]
count_vectorizer = CountVectorizer()
count_vectorizer.fit(text_data)
print(count_vectorizer.vocabulary_)

{'나는': 3, '배가': 8, '고프다': 1, '너는': 5, '고프냐': 0, '내일': 4, '점심': 9, '뭐먹지': 7, '공부': 2, '해야겠다': 10, '해야지': 11, '먹고': 6}


In [131]:
print(count_vectorizer.transform(text_data).toarray())

[[1 1 0 1 0 1 0 0 2 0 0 0]
 [0 0 0 0 1 1 0 2 0 1 0 0]
 [0 0 2 0 2 0 0 0 0 0 1 1]
 [0 0 2 0 0 1 1 0 0 1 0 2]]


In [133]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidt_vectorizer = TfidfVectorizer()
tfidt_vectorizer.fit(text_data)
print(tfidt_vectorizer.transform(text_data).toarray())
print(tfidt_vectorizer.vocabulary_)

[[0.36742339 0.36742339 0.         0.36742339 0.         0.23452159
  0.         0.         0.73484678 0.         0.         0.        ]
 [0.         0.         0.         0.         0.33166972 0.26851522
  0.         0.84136197 0.         0.33166972 0.         0.        ]
 [0.         0.         0.61404114 0.         0.61404114 0.
  0.         0.         0.         0.         0.38941666 0.30702057]
 [0.         0.         0.59590882 0.         0.         0.24121977
  0.37791739 0.         0.         0.29795441 0.         0.59590882]]
{'나는': 3, '배가': 8, '고프다': 1, '너는': 5, '고프냐': 0, '내일': 4, '점심': 9, '뭐먹지': 7, '공부': 2, '해야겠다': 10, '해야지': 11, '먹고': 6}


In [134]:
cv = count_vectorizer.transform(text_data).toarray()
tv = tfidt_vectorizer.transform(text_data).toarray() # cv, tv의 벡터 공간은 동일해야한다

In [135]:
print(cv, tv)

[[1 1 0 1 0 1 0 0 2 0 0 0]
 [0 0 0 0 1 1 0 2 0 1 0 0]
 [0 0 2 0 2 0 0 0 0 0 1 1]
 [0 0 2 0 0 1 1 0 0 1 0 2]] [[0.36742339 0.36742339 0.         0.36742339 0.         0.23452159
  0.         0.         0.73484678 0.         0.         0.        ]
 [0.         0.         0.         0.         0.33166972 0.26851522
  0.         0.84136197 0.         0.33166972 0.         0.        ]
 [0.         0.         0.61404114 0.         0.61404114 0.
  0.         0.         0.         0.         0.38941666 0.30702057]
 [0.         0.         0.59590882 0.         0.         0.24121977
  0.37791739 0.         0.         0.29795441 0.         0.59590882]]


In [143]:
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances, manhattan_distances

points = np.array([[1,1], [5,5]])
euclidean_distances(points[0:1], points[1:2]) # 멀먼 멀 수록 숫자가 큼

array([[5.65685425]])

In [144]:
cosine_similarity(points[0:1], points[1:2]) # 유사하면 1, 다르면 0 에 가깝게 나옴

array([[1.]])

In [146]:
manhattan_distances(points[0:1], points[1:2]) # 제곱에 루트 씌우는게 아니고. x속성 차, y속성 차 를 각각 절대값을 더함 (4+4)

array([[8.]])

In [151]:
print(euclidean_distances(cv[0:1], cv[1:2])) # 1번 문장과 2번 문장
print(euclidean_distances(cv[0:1], cv[2:3])) # 1번 문장과 3번 문장
print(euclidean_distances(tv[0:1], tv[1:2]))
print(euclidean_distances(tv[0:1], tv[2:3]))
print(cosine_similarity(cv[0:1], cv[1:2])) # 1번 문장과 2번 문장
print(cosine_similarity(cv[0:1], cv[2:3])) # 1번 문장과 3번 문장
print(cosine_similarity(tv[0:1], tv[1:2]))
print(cosine_similarity(tv[0:1], tv[2:3]))

[[3.60555128]]
[[4.24264069]]
[[1.3689612]]
[[1.41421356]]
[[0.13363062]]
[[0.]]
[[0.06297262]]
[[0.]]


In [148]:
print(cv[0:1])

[[1 1 0 1 0 1 0 0 2 0 0 0]]
